In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
from scipy.linalg import svd
from scipy.linalg import cho_factor, cho_solve

In [ ]:
starry.config.lazy = False

In [ ]:
map = starry.Map(10, inc=60, lazy=False)
theta = np.linspace(0, 360, 1000, endpoint=False)
X = map.design_matrix(theta=theta)

In [ ]:
np.random.seed(0)
y = 1e-2 * np.random.randn(map.Ny)

In [ ]:
ferr = 1e-4
flux = X.dot(y) + ferr * np.random.randn(len(theta))

In [ ]:
plt.plot(theta, flux);

In [ ]:
mu = np.zeros(map.Ny)
L = 1e-4 * np.eye(map.Ny)

In [ ]:
map.set_data(flux, C=ferr ** 2)
map.set_prior(mu=mu, L=L)
yhat, cho_ycov = map.solve(design_matrix=X)
cho_ycov = np.tril(cho_ycov)

In [ ]:
# Pre-compute
U, s, VT = svd(X)
M = np.linalg.matrix_rank(X)
B = np.array(VT[:M]).T

# Solve
xhat, cho_xcov = starry.linalg.solve(
    X.dot(B), flux, C=ferr ** 2, mu=B.T.dot(mu), L=B.T.dot(L).dot(B)
)
cho_xcov = np.tril(cho_xcov)

# Transform the mean
yhat2 = B.dot(xhat)

# Transform the covariance (CHECK)
xcov = cho_xcov.dot(cho_xcov.T)
invycov = B.dot(np.linalg.inv(xcov)).dot(B.T)
ycov = np.linalg.inv(invycov + np.linalg.inv(L))
cho_ycov2, _ = cho_factor(ycov, lower=True)
cho_ycov2 = np.tril(cho_ycov2)

In [ ]:
plt.plot(yhat, lw=3)
plt.plot(yhat2, lw=1);

In [ ]:
np.allclose(yhat, yhat2)

In [ ]:
np.allclose(cho_ycov, cho_ycov2, atol=1e-4)

In [ ]:
plt.imshow(np.log10(np.abs(cho_ycov)), vmin=-15, vmax=0)
plt.colorbar();

In [ ]:
plt.imshow(np.log10(np.abs(cho_ycov2)), vmin=-15, vmax=0)
plt.colorbar();